# Predict Student Performance

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

2025-02-20 22:51:21.078585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-20 22:51:21.567293: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-20 22:51:21.567331: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-02-20 22:51:21.652653: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 22:51:23.457148: W tensorflow/stream_executor/platform/de

## Set variable

In [2]:
PIPELINE_NAME = "student-performance-pipeline"
SCHEMA_PIPELINE_NAME = "student-performance-tfdv-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

DATA_ROOT = "data"

In [3]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Pengolahan Data

### Data Ingestion

In [4]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [5]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 45
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
interactive_context.show(example_gen.outputs['examples'])

### Data Validation

#### Summary Statistic

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 46
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

#### Data Schema

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 47
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Attendance (%)',FLOAT,required,,-
'Grades',FLOAT,required,,-
'Sleep Hours',FLOAT,required,,-
'Socioeconomic Score',FLOAT,required,,-
'Study Hours',FLOAT,required,,-


#### Anomali pada dataset

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 48
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

### Data Preprocessing

In [13]:
TRANSFORM_MODULE_FILE = "student_performance_transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
    
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}

    attendance = inputs['Attendance (%)']
    outputs['normalized_attendance'] = tft.scale_to_z_score(attendance)

    grades = inputs['Grades']
    outputs['normalized_grades'] = tft.scale_to_0_1(grades)

    sleep_hours = inputs['Sleep Hours']
    outputs['normalized_sleep_hours'] = tft.scale_to_0_1(sleep_hours)

    socioeconomic_score = inputs['Socioeconomic Score']
    outputs['normalized_socioeconomic_score'] = tft.scale_to_0_1(socioeconomic_score)

    study_hours = inputs['Study Hours']
    outputs['normalized_study_hours'] = tft.scale_to_0_1(study_hours)
    
    return outputs

Overwriting student_performance_transform.py


In [15]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_performance_trainer.py -> build/lib
copying student_performance_transform.py -> build/lib
installing to /tmp/tmp3tqtcf6d
running install
running install_lib
copying build/lib/student_performance_trainer.py -> /tmp/tmp3tqtcf6d
copying build/lib/student_performance_transform.py -> /tmp/tmp3tqtcf6d
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp3tqtcf6d/tfx_user_code_Transform-0.0+1e93815c2fa366f6

/home/ardi/miniconda3/envs/ml-predict-student-performance/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Transform-0.0+1e93815c2fa366f6e09a27139ffa2a1813ffe9023a558371182d28e524b2833d-py3-none-any.whl
Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Transform-0.0+1e93815c2fa366f6e09a27139ffa2a1813ffe9023a558371182d28e524b2833d-py3-none-any.whl
Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Transform-0.0+1e93815c2fa366f6e09a27139ffa2a1813ffe9023a558371182d28e524b2833d-py3-none-any.whl
Instructions for updating:
Use ref() instead.


2025-02-20 22:51:42.714678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-20 22:51:42.714978: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-20 22:51:42.715027: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ardi-device): /proc/driver/nvidia/version does not exist
2025-02-20 22:51:42.715963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Transform/transform_graph/49/.temp_path/tftransform_tmp/c3e13aa017d74b258e8dca9b30649ddf/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 49
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Pengembangan dan Validasi Model

### Model

In [16]:
TRAINER_MODULE_FILE="student_performance_trainer.py"

In [52]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam.impl as tft_beam
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx_bsl.public import tfxio
from tfx.examples.penguin import penguin_utils_base as base

# Feature keys
FEATURE_KEYS = [
    'normalized_attendance',
    'normalized_sleep_hours',
    'normalized_socioeconomic_score',
    'normalized_study_hours'
]

LABEL_KEY = 'normalized_grades'

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32) -> tf.data.Dataset:
    # Get post_transform feature spec
    transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY)

    return dataset

def build_model():
    """Membangun model Keras."""
    inputs = {key: tf.keras.layers.Input(shape=(1,), name=key) for key in FEATURE_KEYS}
    x = tf.keras.layers.Concatenate()(list(inputs.values()))
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

def run_fn(fn_args: FnArgs):
    """Fungsi utama Trainer untuk melatih model."""
    
    # Membangun model
    model = build_model()

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Membaca dataset
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, 10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, 10)

    print(train_dataset.element_spec)
    # Callback untuk menyimpan model terbaik
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        save_best_only=True
    )

    # Melatih model
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=10,
        callbacks=[model_checkpoint]
    )

    # Menyimpan model akhir
    model.save(fn_args.serving_model_dir, save_format='tf')


Overwriting student_performance_trainer.py


In [53]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE), # Path ke file Python yang berisi fungsi run_fn
    examples=transform.outputs['transformed_examples'], # Contoh dari komponen ExampleGen atau Transform
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'], # Skema data
    train_args=trainer_pb2.TrainArgs(splits=['train']), # Training arguments (misalnya, num_steps)
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),# Evaluation arguments (misalnya, num_steps)
)

interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_performance_trainer.py -> build/lib
copying student_performance_transform.py -> build/lib
installing to /tmp/tmpsucoxjaz
running install
running install_lib
copying build/lib/student_performance_trainer.py -> /tmp/tmpsucoxjaz
copying build/lib/student_performance_transform.py -> /tmp/tmpsucoxjaz
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpsucoxjaz/tfx_user_code_Trainer-0.0+c125e6d6ece433c8f015db47b00e7f6a2c

/home/ardi/miniconda3/envs/ml-predict-student-performance/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Trainer-0.0+c125e6d6ece433c8f015db47b00e7f6a2cf0fd9bcef2eb0f79f365f5b6c35fdb-py3-none-any.whl
({'normalized_attendance': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'normalized_sleep_hours': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'normalized_socioeconomic_score': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'normalized_study_hours': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))
Epoch 1/10
    344/Unknown - 2s 2ms/step - loss: 0.0091 - mae: 0.0699INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


344/344 [==============================] - 12s 32ms/step - loss: 0.0091 - mae: 0.0699 - val_loss: 0.0032 - val_mae: 0.0403
Epoch 2/10
323/344 [===========================>..] - ETA: 0s - loss: 0.0030 - mae: 0.0408INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


...

344/344 [==============================] - 2s 6ms/step - loss: 4.7050e-04 - mae: 0.0166 - val_loss: 4.9920e-04 - val_mae: 0.0165
Epoch 9/10
344/344 [==============================] - 1s 3ms/step - loss: 4.2824e-04 - mae: 0.0160 - val_loss: 5.5175e-04 - val_mae: 0.0176
Epoch 10/10
339/344 [============================>.] - ETA: 0s - loss: 3.9232e-04 - mae: 0.0154INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


344/344 [==============================] - 2s 5ms/step - loss: 3.9144e-04 - mae: 0.0154 - val_loss: 4.4418e-04 - val_mae: 0.0158
INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/66/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 66
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Analisis dan Validasi Model

#### Komponen Resolver

In [54]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 67
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

#### Komponen Evaluator

In [55]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='is_sarcastic')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [56]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

                the inputs are:
 [b'\n\x83\x01\n\x17\n\x0bStudy Hours\x12\x08\x12\x06\n\x04\xcd\xccL@\n\x17\n\x0bSleep Hours\x12\x08\x12\x06\n\x04\xcd\xcc\xbc@\n\x1f\n\x13Socioeconomic Score\x12\x08\x12\x06\n\x04\x89\x0c[?\n\x1a\n\x0eAttendance (%)\x12\x08\x12\x06\n\x04\x00\x00\\B\n\x12\n\x06Grades\x12\x08\x12\x06\n\x04\x00\x00\x0cB'].
                The input_specs are:
 {'normalized_attendance': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_attendance'), 'normalized_sleep_hours': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_sleep_hours'), 'normalized_socioeconomic_score': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_socioeconomic_score'), 'normalized_study_hours': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_study_hours')}.. Attempting to run batch through serially. Note that this will significantly affect the performance.


ValueError: Fail to call signature func with signature_name: serving_default.
                the inputs are:
 [b'\n\x83\x01\n\x17\n\x0bStudy Hours\x12\x08\x12\x06\n\x04\xcd\xccL@\n\x17\n\x0bSleep Hours\x12\x08\x12\x06\n\x04\xcd\xcc\xbc@\n\x1f\n\x13Socioeconomic Score\x12\x08\x12\x06\n\x04\x89\x0c[?\n\x1a\n\x0eAttendance (%)\x12\x08\x12\x06\n\x04\x00\x00\\B\n\x12\n\x06Grades\x12\x08\x12\x06\n\x04\x00\x00\x0cB'].
                The input_specs are:
 {'normalized_attendance': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_attendance'), 'normalized_sleep_hours': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_sleep_hours'), 'normalized_socioeconomic_score': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_socioeconomic_score'), 'normalized_study_hours': TensorSpec(shape=(None, 1), dtype=tf.float32, name='normalized_study_hours')}. [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/ExtractPredictions/Inference']